In [1]:
# Basic Python libraries
import numpy as np
import pandas as pd

In [2]:
#Visualization Libraries
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
#To plot the graph embedded in the notebook
%matplotlib inline

In [4]:
#imports from sklearn library

from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error

In [5]:
#loading the dataset direclty from sklearn
boston = datasets.load_boston()